In [ ]:
import tqdm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler#, MinMaxScaler
scaler = StandardScaler()
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV

In [ ]:
OUTCOME = 'T1D_STRICT'
SEED = 4
FOLD_NUM = 10
covariates = ['PC'+str(i) for i in range(1, 11)] + ['sex', 'age']
eps_sig = ['E4_GRAVES_STRICT','D3_ANAEMIA_B12_DEF','E4_HYTHY_AI_STRICT',
           'K11_COELIAC','M13_SJOGREN','M13_RHEUMA','T1D_STRICT']
eps_dict = {
    'D3_AIHA_OTHER':'Autoimmune hemolytic anemia',
    'D3_ALLERGPURPURA':'Allergic purpura',
    'D3_ANAEMIA_B12_DEF':'Vitamin B12 deficiency anaemia',
    'D3_ITP':'Idiopathic thrombocytopenic purpura',
    'CHIRBIL_PRIM':'Primary biliary cholangitis',
    'K11_COELIAC':'Coeliac disease',
    'K11_IBD':'Inflammatory bowel disease',
    'N14_IGA_NEPHROPATHY':'IgA nephropathy',
    'M13_MCTD':'Mixed connective tissue disease',
    'M13_RHEUMA':'Rheumatoid arthritis',
    'M13_SJOGREN':'Sjögren syndrome',
    'M13_SYSTSLCE':'Systemic sclerosis',
    'M13_WEGENER':'Wegener granulomatosis',
    'SLE_FG':'Systemic lupus erythematosus',
    'G6_GUILBAR':'Guillain-Barre syndrome',
    'G6_MS':'Multiple Sclerosis',
    'G6_MYASTHENIA':'Myasthenia gravis',
    'L12_ALOPECAREATA':'Alopecia areata',
    'L12_PSORIASIS':'Psoriasis',
    'L12_VITILIGO':'Vitiligo',
    'E4_ADDISON':'Adrenocortical insufficiency',
    'E4_GRAVES_STRICT':'Autoimmune hyperthyroidism',
    'E4_HYTHY_AI_STRICT':'Autoimmune hypothyroidism',
    'T1D_STRICT':'Type 1 diabetes'
}

In [ ]:
genes = ['A*01:01', 'A*01:02', 'A*02:01', 'A*02:02', 'A*02:03', 'A*02:05', 'A*02:06', 'A*02:07', 'A*02:17', 'A*03:01', 'A*11:01',
 'A*23:01', 'A*24:02', 'A*25:01', 'A*26:01', 'A*29:01', 'A*29:02', 'A*30:01', 'A*30:02', 'A*31:01', 'A*32:01', 'A*33:01',
 'A*33:03', 'A*33:05', 'A*68:01', 'A*69:01', 'A*68167', 'C*01:02', 'C*02:02', 'C*03:02', 'C*03:03', 'C*03:04', 'C*04:01',
 'C*04:06', 'C*05:01', 'C*06:02', 'C*07:01', 'C*07:02', 'C*07:04', 'C*08:02', 'C*12:02', 'C*12:03', 'C*14:02', 'C*15:02',
 'C*15:05', 'C*16:01', 'C*16:02', 'C*17:01', 'C*17:03', 'C*03327', 'B*07:01', 'B*07:02', 'B*08:01', 'B*13:01', 'B*13:02',
 'B*14:01', 'B*14:02', 'B*15:01', 'B*15:16', 'B*15:17', 'B*18:01', 'B*27:02', 'B*27:05', 'B*35:01', 'B*35:02', 'B*35:03',
 'B*35:08', 'B*37:01', 'B*38:01', 'B*39:01', 'B*39:06', 'B*39:24', 'B*40:01', 'B*40:02', 'B*41:01', 'B*41:02', 'B*44:02',
 'B*44:03', 'B*44:27', 'B*45:01', 'B*46:01', 'B*47:01', 'B*49:01', 'B*50:01', 'B*51:01', 'B*52:01', 'B*55:01', 'B*56:01',
 'B*57:01', 'B*58:01', 'DRB3*01:01', 'DRB4*01:01', 'DRB5*01:01', 'DRB3*02:02', 'DRB4*01:03', 'DRB5*01:02', 'DRB3*03:01',
 'DRB4*01:03N', 'DRB5*02:02', 'DRB1*01:01', 'DRB1*01:02', 'DRB1*01:03', 'DRB1*03:01', 'DRB1*04:01', 'DRB1*04:02', 'DRB1*04:03',
 'DRB1*04:04', 'DRB1*04:05', 'DRB1*04:07', 'DRB1*04:08', 'DRB1*07:01', 'DRB1*07:03', 'DRB1*08:01', 'DRB1*08:02', 'DRB1*08:03',
 'DRB1*09:01', 'DRB1*10:01', 'DRB1*11:01', 'DRB1*11:03', 'DRB1*11:04', 'DRB1*12:01', 'DRB1*13:01', 'DRB1*13:02', 'DRB1*13:03',
 'DRB1*13:05', 'DRB1*13:32', 'DRB1*14:01', 'DRB1*14:02', 'DRB1*14:54', 'DRB1*15:01', 'DRB1*15:02', 'DRB1*16:01', 'DQA1*03:02',
 'DQA1*01:01', 'DQA1*03:03', 'DQA1*01:02', 'DQA1*04:01', 'DQA1*01:03', 'DQA1*04:02', 'DQA1*01:04', 'DQA1*05:01', 'DQA1*01:05',
 'DQA1*05:03', 'DQA1*02:01', 'DQA1*05:05', 'DQA1*03:01', 'DQA1*06:01', 'DQB1*02:01', 'DQB1*02:02', 'DQB1*03:01', 'DQB1*03:02',
 'DQB1*03:03', 'DQB1*03:04', 'DQB1*03:05', 'DQB1*04:02', 'DQB1*05:01', 'DQB1*05:02', 'DQB1*05:03', 'DQB1*06:01', 'DQB1*06:02',
 'DQB1*06:03', 'DQB1*06:04', 'DQB1*06:09', 'DPB1*01:01', 'DPB1*02:01', 'DPB1*02:02', 'DPB1*03:01', 'DPB1*04:01', 'DPB1*04:02',
 'DPB1*04.02', 'DPB1*05:01', 'DPB1*06:01', 'DPB1*09:01', 'DPB1*10:01', 'DPB1*11:01', 'DPB1*13:01', 'DPB1*14:01', 'DPB1*15:01',
 'DPB1*16:01', 'DPB1*17:01', 'DPB1*19:01', 'DPB1*20:01', 'DPB1*23:01', 'DPB1*25:01', 'DPB1*31:01', 'DPB1*34:01', 'DPB1*105:01']

In [ ]:
hla_df = pd.read_csv('hla_df_all.csv')
hla_df = hla_df.fillna(0.0)
hla_df.T1D_STRICT.value_counts()

In [ ]:
def regression(ep, col_name, hla_data):
    
    y = hla_data[ep]
    y.index = range(len(y))
    
    x0 = sm.add_constant(hla_data[covariates])
    model = sm.Logit(y, x0).fit(disp=0)
    r_squared_0 = float(model.summary2().tables[0].iloc[0,3])
    
    x1 = sm.add_constant(hla_data[[col_name]+covariates])
    model = sm.Logit(y, x1).fit(disp=0)
    r_squared_1 = float(model.summary2().tables[0].iloc[0,3])
    r_squared_delta = r_squared_1 - r_squared_0
    
    res = model.summary2().tables[1]
    coef_025 = res.loc[col_name,'[0.025']
    coef_975 = res.loc[col_name,'0.975]']
    pval = res.loc[col_name,'P>|z|']
    se = res.loc[col_name,'Std.Err.']
    coef = res.loc[col_name,'Coef.']
    
    return [ep, coef, se, pval, coef_025, coef_975, r_squared_0, r_squared_1, r_squared_delta]


def model_loop(ep, results, eps_list, col_name, hla_data):
    # hla_data = hla_data[~hla_data[ep].isna()]
    hla_data[ep] = hla_data[ep].astype(int)
    for i in eps_list:
        # hla_data = hla_data[~hla_data[i].isna()]
        hla_data[i] = hla_data[i].astype(int)
        hla_data.index = range(len(hla_data))
        n_ep1 = len(hla_data[hla_data[ep] == 1])
        n_ep2 = len(hla_data[hla_data[i] == 1])
        n_both = len(hla_data[(hla_data[ep] == 1)&(hla_data[i] == 1)])
        res = regression(i, col_name, hla_data)
        if res:
            results = results.append(pd.Series([ep]+res+[n_ep1, n_ep2, n_both, col_name], index=results.columns), ignore_index=True)
    return results

In [ ]:
columns=['ep1','ep2', 'coef', 'se', 'pval', 'beta_025', 'beta_975', 'r_squared_0', 'r_squared_1', 'r_squared_delta', 'n_ep1', 'n_ep2', 'n_both', 'method']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
event_path = '/finngen/library-red/finngen_R10/phenotype_1.0/data/finngen_R10_endpoint_longitudinal_1.0.txt.gz'
events = pd.read_csv(event_path, sep='\t')
df_events_sub = events[events.ENDPOINT == OUTCOME]
df_events_sub = df_events_sub.drop_duplicates(['FINNGENID'])
hla_df = hla_df.merge(df_events_sub[['FINNGENID', 'EVENT_AGE']].rename(columns={'EVENT_AGE': 't1d_onset', 'FINNGENID': 'finngen_id'}), 
                              'left', on='finngen_id')

fam_path = '/finngen/library-red/finngen_R9/kinship_1.0/data/finngen_R9_pedigree.fam'
fam = pd.read_csv(fam_path, sep='\t', header=None)
fam.columns = ['family_id', 'finngen_id', 'father_id', 'mother_id', 'sex', 'phenotype']

In [ ]:
fam_fa = fam[fam.father_id.isin(hla_df.finngen_id)]  # 23138
fam_mo = fam[fam.mother_id.isin(hla_df.finngen_id)]  # 26891

In [ ]:
fam_pool = hla_df[hla_df.finngen_id.isin(list(set(fam_fa.finngen_id.tolist()+fam_fa.father_id.tolist()+fam_mo.finngen_id.tolist()+fam_mo.mother_id.tolist())))] # 73391
individuals = hla_df[~hla_df.finngen_id.isin(fam_pool.finngen_id)] # 303892

In [ ]:
families = hla_df[hla_df.finngen_id.isin(list(set(fam_fa.finngen_id.tolist()+fam_mo.finngen_id.tolist())))] # 39367
# removes individuals who are parents
families = families[~families.finngen_id.isin(list(set(fam_fa.father_id.tolist()+fam_mo.mother_id.tolist())))] # 36585  delta = 2782

In [ ]:
individuals_shuffled = individuals.sample(frac=1, random_state=SEED) 
individuals_shuffled.index = range(len(individuals_shuffled))
individuals_n_folds = np.array_split(individuals_shuffled, FOLD_NUM) # shuffle

families_shuffled = families.sample(frac=1, random_state=SEED) 
families_shuffled.index = range(len(families_shuffled))
families_n_folds = np.array_split(families_shuffled, FOLD_NUM) # shuffle

In [ ]:
whole_data = pd.concat([individuals_shuffled, families_shuffled])
whole_data = whole_data.sample(frac=1, random_state=SEED) 
whole_data.index = range(len(whole_data))

In [ ]:
alphas = np.logspace(start=0,stop=5, num=100)
optimal_alphas = {}

for endpoint in tqdm.tqdm(list(eps_dict.keys())):
    ep_counts = whole_data[endpoint].value_counts()
    weights = {0:ep_counts[1]/ep_counts[0], 1:1.}
    
    ridge_m = RidgeClassifierCV(alphas, class_weight=weights, store_cv_values=True)
    ridge_m = ridge_m.fit(whole_data[genes+covariates], whole_data[endpoint])
    optimal_alphas[endpoint] = ridge_m.alpha_

In [ ]:
# step 1: cv ridge classifier for obtaining prs
# optimal alphas

res_n_folds = pd.DataFrame(columns=columns)

for endpoint in tqdm.tqdm(list(eps_dict.keys())):
    for num in range(FOLD_NUM):
        test_df = pd.concat([individuals_n_folds[num], families_n_folds[num]])
        test_df.index = range(len(test_df))
        train_df = whole_data[~whole_data.finngen_id.isin(test_df.finngen_id)]
        train_df.index = range(len(train_df))
        
        ep_counts = train_df[endpoint].value_counts()
        weights = {0:ep_counts[1]/ep_counts[0], 1:1.}

        model = RidgeClassifier(class_weight=weights, alpha=optimal_alphas[endpoint], random_state=4)
        # model = RidgeClassifier(alpha=optimal_alphas[endpoint], random_state=4)
        model.fit(train_df[genes+covariates], train_df[endpoint])
        prs = test_df[genes].to_numpy() @ model.coef_[0,:len(genes)].T
        prs_col_name = endpoint+'_'+str(num)
        test_df[prs_col_name] = scaler.fit_transform(prs.reshape(-1,1))
        res_n_folds = model_loop(endpoint, res_n_folds, [OUTCOME], prs_col_name, test_df)
        res_n_folds = model_loop(endpoint, res_n_folds, [endpoint], prs_col_name, test_df)

In [ ]:
# step 2: meta-analysis weights in R
res_n_folds.to_csv('stats_n_folds_m2.csv', index=None)

In [ ]:
# step 3: plot meta results
meta_res = pd.read_csv('meta_res_m2.csv')
# meta_res = pd.concat([meta_res.iloc[:,0], meta_res.iloc[:,1:].astype(float)], axis=1)
meta_res['n_ep'] = whole_data[list(eps_dict.keys())].sum().values

In [ ]:
meta_res['r2_1'] = [res_n_folds[(res_n_folds.ep1 == i)&(res_n_folds.ep2 == i)].r_squared_delta.mean() for i in list(eps_dict.keys())]
meta_res['r2_2'] = [res_n_folds[(res_n_folds.ep1 == i)&(res_n_folds.ep2 == OUTCOME)].r_squared_delta.mean() for i in list(eps_dict.keys())]

In [ ]:
meta_res['p_1'] = [res_n_folds[(res_n_folds.ep1 == i)&(res_n_folds.ep2 == i)].pval.mean() for i in list(eps_dict.keys())]
meta_res['p_2'] = [res_n_folds[(res_n_folds.ep1 == i)&(res_n_folds.ep2 == OUTCOME)].pval.mean() for i in list(eps_dict.keys())]